In [1]:
import pandas as pd
import numpy as np

In [40]:
orders = pd.read_csv('../../data/instacart_2017_05_01/orders.csv')
products = pd.read_csv('../../data/instacart_2017_05_01/products.csv')
order_products = pd.read_csv('../../data/instacart_2017_05_01/order_products__prior.csv')

In [3]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## Calculate average number of items per order, by day of week

In [5]:
item_counts = order_products.groupby('order_id').size()

In [6]:
orders_indexed = orders.set_index('order_id')

In [7]:
order_items = pd.concat([orders_indexed, item_counts], axis=1)

In [8]:
new_columns = order_items.columns.values
new_columns[6] = 'item_count'
order_items.columns = new_columns

In [9]:
order_items.head()

,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,item_count
order_id,,,,,,,
1,112108,train,4,4,10,9.0,NaN
2,202279,prior,3,5,9,8.0,9.0
3,205970,prior,16,5,17,12.0,8.0
4,178520,prior,36,1,9,7.0,13.0
5,156122,prior,42,6,16,9.0,26.0


In [39]:
order_items.groupby('order_dow').mean()['item_count']

order_dow
0    11.132983
1    10.177484
2     9.543501
3     9.321331
4     9.439436
5     9.881950
6    10.744480
Name: item_count, dtype: float64

# Most popular item by day of order

In [10]:
orders_indexed.order_dow.head()

order_id
2539329    2
2398795    3
473747     3
2254736    4
431534     4
Name: order_dow, dtype: int64

In [14]:
order_dow = pd.DataFrame({'order_dow': orders_indexed.order_dow})

In [15]:
order_dow.head()

,order_dow
order_id,
2539329,2
2398795,3
473747,3
2254736,4
431534,4


In [21]:
order_products_dow = order_products.merge(order_dow, left_on='order_id', right_index=True)

In [22]:
order_products_dow.head()

,order_id,product_id,add_to_cart_order,reordered,order_dow
0,2,33120,1,1,5
1,2,28985,2,1,5
2,2,9327,3,0,5
3,2,45918,4,1,5
4,2,30035,5,0,5


In [30]:
product_counts = order_products_dow.groupby(['order_dow','product_id']).count()['order_id']

In [34]:
product_counts.head()

order_dow  product_id
0          1             201
           2              17
           3              50
           4              74
           5               4
Name: order_id, dtype: int64

In [42]:
product_counts.groupby(level=0).nlargest(3)

order_dow  order_dow  product_id
0          0          24852         96769
                      13176         71493
                      21903         54914
1          1          24852         87787
                      13176         69726
                      21137         46361
2          2          24852         59881
                      13176         50271
                      21137         32394
3          3          24852         53327
                      13176         45491
                      21137         30552
4          4          24852         52497
                      13176         44604
                      21137         30676
5          5          24852         58990
                      13176         48618
                      21137         34146
6          6          24852         63314
                      13176         49247
                      21137         36723
Name: order_id, dtype: int64

In [46]:
products[products.product_id == 21903]

,product_id,product_name,aisle_id,department_id
21902,21903,Organic Baby Spinach,123,4


In [29]:
order_products_dow.head()

,order_id,product_id,add_to_cart_order,reordered,order_dow
0,2,33120,1,1,5
1,2,28985,2,1,5
2,2,9327,3,0,5
3,2,45918,4,1,5
4,2,30035,5,0,5
